In [ ]:
import os
import numpy as np
import pandas as pd
import urllib
import scipy
from sklearn.datasets import fetch_openml
import sklearn.datasets
from scipy.special import expit

################# Analytic Gaussian Mechanism #################
"""
    Taken from the official repository of Balle and Wang, ICML'18
    https://github.com/BorjaBalle/analytic-gaussian-mechanism
"""

from math import exp, sqrt
from scipy.special import erf

def calibrateAnalyticGaussianMechanism(epsilon, delta, GS, tol = 1.e-12):
    """ Calibrate a Gaussian perturbation for differential privacy using the analytic Gaussian mechanism of [Balle and Wang, ICML'18]

    Arguments:
    epsilon : target epsilon (epsilon > 0)
    delta : target delta (0 < delta < 1)
    GS : upper bound on L2 global sensitivity (GS >= 0)
    tol : error tolerance for binary search (tol > 0)

    Output:
    sigma : standard deviation of Gaussian noise needed to achieve (epsilon,delta)-DP under global sensitivity GS
    """

    def Phi(t):
        return 0.5*(1.0 + erf(float(t)/sqrt(2.0)))

    def caseA(epsilon,s):
        return Phi(sqrt(epsilon*s)) - exp(epsilon)*Phi(-sqrt(epsilon*(s+2.0)))

    def caseB(epsilon,s):
        return Phi(-sqrt(epsilon*s)) - exp(epsilon)*Phi(-sqrt(epsilon*(s+2.0)))

    def doubling_trick(predicate_stop, s_inf, s_sup):
        while(not predicate_stop(s_sup)):
            s_inf = s_sup
            s_sup = 2.0*s_inf
        return s_inf, s_sup

    def binary_search(predicate_stop, predicate_left, s_inf, s_sup):
        s_mid = s_inf + (s_sup-s_inf)/2.0
        while(not predicate_stop(s_mid)):
            if (predicate_left(s_mid)):
                s_sup = s_mid
            else:
                s_inf = s_mid
            s_mid = s_inf + (s_sup-s_inf)/2.0
        return s_mid

    delta_thr = caseA(epsilon, 0.0)

    if (delta == delta_thr):
        alpha = 1.0

    else:
        if (delta > delta_thr):
            predicate_stop_DT = lambda s : caseA(epsilon, s) >= delta
            function_s_to_delta = lambda s : caseA(epsilon, s)
            predicate_left_BS = lambda s : function_s_to_delta(s) > delta
            function_s_to_alpha = lambda s : sqrt(1.0 + s/2.0) - sqrt(s/2.0)

        else:
            predicate_stop_DT = lambda s : caseB(epsilon, s) <= delta
            function_s_to_delta = lambda s : caseB(epsilon, s)
            predicate_left_BS = lambda s : function_s_to_delta(s) < delta
            function_s_to_alpha = lambda s : sqrt(1.0 + s/2.0) + sqrt(s/2.0)

        predicate_stop_BS = lambda s : abs(function_s_to_delta(s) - delta) <= tol

        s_inf, s_sup = doubling_trick(predicate_stop_DT, 0.0, 1.0)
        s_final = binary_search(predicate_stop_BS, predicate_left_BS, s_inf, s_sup)
        alpha = function_s_to_alpha(s_final)

    sigma = alpha*GS/sqrt(2.0*epsilon)

    return sigma

################# Analytic solver for the noise required for the GaussMix mechanism #################
"""
    Correspond to the different calculations done in:
"""

# Find noise required for Gaussian mechanism with composition over T steps
# by performing the exact conversion from RDP to DP
def objective_Gaussian_composition(alpha, sigma, delta,C_max, T):
    if alpha <= 1:
        return np.inf  # Penalize out-of-bound values

    term1 = alpha * (C_max**2) / (2.0 * sigma**2)
    term2 = (np.log(1.0 / delta) + (alpha - 1)*np.log(1-1/alpha) - np.log(alpha)) / (alpha - 1)
    return T * term1 + term2

def objective_func_full_composition(alpha, k, gamma, delta, inflation_norm, T):
    if alpha <= 1 or alpha >= gamma/(1 + inflation_norm):
        return np.inf  # Penalize out-of-bound values

    term1 = (k * alpha) / (2 * (alpha - 1)) * np.log(1.0 - (1.0 + inflation_norm) / (gamma))
    term2 = - (k / (2 * (alpha - 1))) * np.log(1 - (alpha*(1 + inflation_norm)) / gamma)
    term3 = (np.log(1.0 / delta) + (alpha - 1)*np.log(1-1/alpha) - np.log(alpha)) / (alpha - 1)
    term4 = np.sqrt(2.0 * np.log(1.25/delta)) / (gamma/np.sqrt(k))
    return T * (term1 + term2) + term3 + term4

def objective_func_full(alpha, k, gamma, delta, inflation_norm):
    if alpha <= 1 or alpha >= gamma/(1+inflation_norm):
        return np.inf  # Penalize out-of-bound values

    term1 = (k * alpha) / (2 * (alpha - 1)) * np.log(1.0 - (1.0 + inflation_norm) / (gamma))
    term2 = - (k / (2 * (alpha - 1))) * np.log(1 - (alpha*(1 + inflation_norm)) / gamma)
    term3 = (np.log(1.0 / delta) + (alpha - 1)*np.log(1-1/alpha) - np.log(alpha)) / (alpha - 1)
    term4 = np.sqrt(2.0 * np.log(1.25/delta)) / (gamma/np.sqrt(k))
    return term1 + term2 + term3 + term4

def solve_gamma_renyi_full_composition(init_gamma, k, target_delta, target_epsilon, inflation_norm, T):
    # Solve for the \gamma required for target \eps,\delta DP after composing T steps
    # Define binary search bounds
    left, right = init_gamma / 500000.0, 500000.0*init_gamma
    best_gamma = right  # Default to upper bound in case no solution is found

    # The bound is for composition over 3 contributions for \delta.
    # Thus, we divide \delta by 3
    internal_target_delta = target_delta/3.0
    while right - left > 1e-6:  # Precision threshold
        mid_gamma = (left + right) / 2

        # Solve for optimal alpha given the current gamma
        result = scipy.optimize.minimize_scalar(objective_func_full_composition,
                                 bounds=(1 + 1e-5, mid_gamma - 1e-5),
                                 args=(k, mid_gamma, internal_target_delta, inflation_norm, T),
                                 method='bounded') # minimization is between 1 < \alpha < \gamma

        if result.success and result.fun < target_epsilon:
            best_gamma = mid_gamma  # Update best found gamma
            right = mid_gamma  # Search for a smaller gamma
        else:
            left = mid_gamma  # Increase gamma to meet target_epsilon
    return best_gamma

# Solve the required noise for single sketching step by
# performing the full conversion from RenyiDP to DP numerically
def solve_gamma_renyi_full(init_gamma, k, target_delta, target_epsilon, inflation_norm):
    # Define binary search bounds
    left, right = init_gamma / 500000.0, 500000.0*init_gamma
    best_gamma = right  # Default to upper bound in case no solution is found

    # The bound is for composition over 3 contributions for \delta. Thus, we divide \delta by 3
    internal_target_delta = target_delta/3.0
    while right - left > 1e-6:  # Precision threshold
        mid_gamma = (left + right) / 2
        # Solve for optimal alpha given the current gamma
        result = scipy.optimize.minimize_scalar(objective_func_full,
                                 bounds=(1 + 1e-5, mid_gamma - 1e-5),
                                 args=(k, mid_gamma, internal_target_delta, inflation_norm),
                                 method='bounded') # minimization is between 1 < \alpha < \gamma
        if result.success and result.fun < target_epsilon:
            best_gamma = mid_gamma  # Update best found gamma
            right = mid_gamma  # Search for a smaller gamma
        else:
            left = mid_gamma  # Increase gamma to meet target_epsilon
    return best_gamma

# Solve with full conversion from RenyiDP to DP for the Gaussian mechanism
def solve_Gaussian_composition(init_gamma, delta, target_epsilon, C_max, T):
    # Solve for the \gamma required for target \eps,\delta DP after composing T steps
    # Define binary search bounds
    left, right = init_gamma / 500000.0, 500000.0*init_gamma
    best_gamma = right  # Default to upper bound in case no solution is found
    while right - left > 1e-6:  # Precision threshold
        mid_gamma = (left + right) / 2

        # Solve for optimal alpha given the current gamma
        result = scipy.optimize.minimize_scalar(objective_Gaussian_composition,
                                 bounds=(1 + 1e-5, 1e6),
                                 args=(mid_gamma, delta,C_max, T),
                                 method='bounded') # effectively unbounded by the upper limit being very large

        if result.success and result.fun < target_epsilon:
            best_gamma = mid_gamma  # Update best found gamma
            right = mid_gamma  # Search for a smaller gamma
        else:
            left = mid_gamma  # Increase gamma to meet target_epsilon
    return best_gamma


# Load datasets
def GenerateClippingDataset(n):

    # Set parameters
    n_samples = n
    n_features = 1
    layer_sizes = [2**7, 2**7, 2**7] # [2**9, 2**9, 2**9] # [100, 100, 100]
    mu, sigma = 1, 1  # For lognormal distribution

    X = np.random.lognormal(mean=mu, sigma=sigma, size=(n_samples, n_features))

    # Initialize weights and biases for the MLP
    theta = np.random.normal(size=(layer_sizes[2], 1))
    weights = [
        np.random.normal(size=(n_features, layer_sizes[0])),
        np.random.normal(size=(layer_sizes[0], layer_sizes[1])),
        np.random.normal(size=(layer_sizes[1], layer_sizes[2])),
        theta/np.sqrt(np.sum(theta**2))
    ]

    biases = [
        0.001 * np.random.normal(size=(size,)) for size in layer_sizes + [1]
    ]

    # Generate the response variable

    def mlp_forward(X, weights, biases):
        layer_output = X
        for w, b in zip(weights[:-1], biases[:-1]):
            layer_output = expit(np.dot(layer_output, w) + b)
        # Output layer
        output = np.dot(layer_output, weights[-1]) + biases[-1]
        return output, layer_output

    output, mid_X = mlp_forward(X, weights, biases)
    sigma_z = 0.2
    Y = output.flatten() + sigma_z * np.random.normal(size=n_samples)
    Y[np.abs(Y) > 1.0] == 1.0 * np.sign(Y[np.abs(Y) > 1.0])
    C_max = 1.0

    # Apply random permutation and apply train test split
    m, n = mid_X.shape
    p = np.random.permutation(m)
    mid_X = mid_X[p, :]
    Y = Y[p]

    train_size = int(0.8 * n_samples)
    test_size = n_samples - train_size
    X_train = mid_X[:train_size]
    y_train = Y[:train_size]
    X_test = mid_X[train_size:]
    y_test = Y[train_size:]

    # normalize dataset to have power 1
    norm_fact = np.sqrt(np.max(np.sum(X_train**2, 1)))  # np.mean(np.mean(X_train**2))
    X_train = X_train/norm_fact
    X_test = X_test/norm_fact
    n, d = X_train.shape

    # Calculate minimum eigenvalue of X^T X and of (X,Y)^T (X,Y)
    lambda_max = np.max(np.linalg.eigvals(X_train.T @ X_train))
    lambda_min = np.min(np.linalg.eigvals(X_train.T @ X_train))
    y_col = y_train.reshape(-1, 1) if y_train.ndim == 1 else y_train
    XY = np.hstack((X_train, y_col))
    lambda_min_XY = np.real(np.min(np.linalg.eigvals(XY.T @ XY)))

    return C_max, X_train, y_train, X_test, y_test, np.real(lambda_min), np.real(lambda_max), np.real(lambda_min_XY), n, d


# Run for dataset with large condition number

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from tqdm.notebook import tqdm
import random
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")  # suppress all warnings from all modules

# Fix initial seed for reproducibility
random.seed(50)
np.random.seed(50)


############# Hyperparameter T #############
iters_IHM = 3

############# Hyperparameter: k #############
# For IHM, overall k is going to be percentage_k_IHM * max(d, log(4T/varrho))
# For LinearMixing, overall k is going to be percentage_k_Mix * max(d, log(1/varrho))
percentage_k_IHM = 6
percentage_k_Mix = 2.5

############# Hyperparameter: learning rate for DP-GD #############
# Globl learning rate for all the different cases: calibrated offline to provide the best performance among all cases
lr_dp_gd = 0.25

# Number of Monte-Carlo iterations
iters = 30

# Which methods to plot: pick from the next list
# 'AdaSSP'          : AdaSSP
# 'LinearMix'       : Linear Mixing
# 'HessianMix'      : Hessian mixing with iters_IHM iterations
# 'HessianMix_T_1'  : Hessian mixing with a single iteration, with the same k-value as Linear Mixing
# 'DPGD'            : DP-GD

methods_to_plot = ['AdaSSP', 'LinearMix', 'HessianMix']

add_legend_to_plot = False

# epsilon values to run our algorithm
epsilon_values = np.logspace(-1.0, 1.0, 4)  # np.logspace(-1.0, 1.0, 6)

# Start looping over datasets

print('Running clipping experiment')
# Parse dataset
C_max, X_train, y_train,\
    X_test, y_test, \
    lambda_min, lambda_max, lambda_min_XY,\
    n, d = GenerateClippingDataset(n=2**18)

print('=============================')
print('lambda min is ' + str(lambda_min)
      + ', lambda min XY is ' + str(lambda_min_XY))
print('n is ' + str(n) + ', d is ' + str(d))
print('=============================')

# Delta and target varrho
delta_DP = 1/(n**2)
target_varrho = delta_DP/10

# Hyerparameter: tau - eigenvalue threshold constant
tau = np.sqrt(2.0 * np.log(np.max((3.0/delta_DP, 2.0/target_varrho))))
tau_mix = np.sqrt(2.0 * np.log(np.max((4.0/delta_DP, 4.0/target_varrho))))

# Baseline: OLS
lambda_baseline = 1e-16
ols_model = LinearRegression(fit_intercept=False)  # alpha is the regularization strength
ols_model.fit(X_train, y_train)
ridge_y_pred = ols_model.predict(X_test)
baseline_test_mse = np.sum((y_test - ridge_y_pred)**2)
ridge_y_pred_train = ols_model.predict(X_train)
baseline_train_mse = np.sum((y_train - ridge_y_pred_train)**2)
print('============================')
print('baseline test mse is  ' + str(baseline_test_mse/n))
print('baseline train mse is ' + str(baseline_train_mse/n))
print('============================')

# Set k_value
k_val_LinMix = np.max((int(percentage_k_Mix * d), int(percentage_k_Mix * np.log(2.0/target_varrho))))  # target_rho is the failure probability of the random projection
k_val_IHM    = np.max((int(percentage_k_IHM * d), int(percentage_k_IHM * np.log(4.0*iters_IHM/target_varrho))))  # target_rho is the failure probability of the random projection

# Prepare a list to store MSE for each sigma
mse_for_sigmas                    = []
mse_for_sigmas_std                = []
mse_for_sigmas_adassp             = []
mse_for_sigmas_std_adassp         = []
mse_for_sigmas_IHM                = []
mse_for_sigmas_std_IHM            = []
mse_for_sigmas_IHM_T_1            = []
mse_for_sigmas_std_IHM_T_1        = []
mse_for_sigmas_DPGD               = []
mse_for_sigmas_std_DPGD           = []
train_mse_for_sigmas              = []
train_mse_for_sigmas_std          = []
train_mse_for_sigmas_adassp       = []
train_mse_for_sigmas_std_adassp   = []
train_mse_for_sigmas_IHM          = []
train_mse_for_sigmas_std_IHM      = []
train_mse_for_sigmas_IHM_T_1      = []
train_mse_for_sigmas_std_IHM_T_1  = []
train_mse_for_sigmas_DPGD         = []
train_mse_for_sigmas_std_DPGD     = []
condition_number_vec              = []
clipping_perc_vector              = []

# Iterate over different epsilons
for eps_idx, eps in tqdm(enumerate(epsilon_values)):
    print('Started eps = ' + str(eps))

    # Initiate arrays for results of current \eps
    curr_test_mse_linear_mixing  = []
    curr_test_mse_adassp         = []
    curr_test_mse_IHM            = []
    curr_test_mse_IHM_T_1        = []
    curr_test_mse_DP_GD          = []

    curr_train_mse_linear_mixing = []
    curr_train_mse_adassp        = []
    curr_train_mse_IHM           = []
    curr_train_mse_IHM_T_1       = []
    curr_train_mse_DP_GD         = []

    ######################## Compute Noise Values ##########################
    # Baseline for numeric calculations: (\eps, \delta)-DP with classical Gaussian mechanism
    sigma_eps_delta_DP_Gaussian = 2.0 * np.log(1.25/delta_DP) / (eps**2)

    ######################## Linear Mixing ########################
    # Linear mixing: noise for sketching required for target (\eps,\delta)-DP with inflation of (C_max)**2
    gamma_matrix = solve_gamma_renyi_full(init_gamma=sigma_eps_delta_DP_Gaussian, k=k_val_LinMix, target_delta=delta_DP,\
                                                    target_epsilon=eps, inflation_norm=C_max**2)

    # we mutiply by \sqrt{1 + C_max**2} since the sensitivity of the
    # minimal eigenvalue lambda_min_XY is (1 + C_max**2), and \sqrt{\gamma} (and also sigma_eigenval) has units of \sqrt{1.0 + C_max**2}
    sigma_eigenval = np.sqrt(1.0 + C_max**2) * gamma_matrix/np.sqrt(k_val_LinMix)

    ######################## IHM with T=1 ########################
    # Hessian mixing with T=1: noise for sketching required for target (\eps/2,\delta/2)-DP with no norm inflation
    sigma_IHM_T_1_X = solve_gamma_renyi_full(init_gamma=sigma_eps_delta_DP_Gaussian, k=k_val_LinMix, target_delta=3.0*delta_DP/4.0,\
                                                    target_epsilon=eps/2.0, inflation_norm=0.0)
    sigma_eigenval_IHM_T_1 = sigma_IHM_T_1_X/np.sqrt(k_val_LinMix)
    sigma_IHM_T_1_XTY = calibrateAnalyticGaussianMechanism(eps/2.0, delta_DP/4.0, C_max, tol = 1.e-12)

    ######################## IHM with T=iters_IHM ########################
    # Hessian mixing with T=iters_IHM: noise for sketching required for target (\eps/2,\delta/2)-DP with no norm inflation
    gamma_matrix_half_iter = solve_gamma_renyi_full_composition(init_gamma=sigma_eps_delta_DP_Gaussian, k=k_val_IHM, \
                                        target_delta=3.0*delta_DP/4.0, target_epsilon=eps/2.0, inflation_norm=0.0, T=iters_IHM)
    sigma_eigenval_half_iter = gamma_matrix_half_iter/np.sqrt(k_val_IHM)
    sigma_IHM_XTY = calibrateAnalyticGaussianMechanism(eps/2.0, delta_DP/4.0, C_max, tol = 1.e-12)
    sigma_IHM_XTY *= np.sqrt(iters_IHM)

    # sigma_IHM_XTY_analytic_conversion_RDP_to_DP = solve_Gaussian_composition(sigma_eps_delta_DP_Gaussian, delta_DP/4.0, \
    #             eps/2.0, C_max, iters_IHM)

    beta_2 = np.max((0, gamma_matrix_half_iter - lambda_min))
    condition_number = np.sqrt((lambda_max + beta_2)/(lambda_min + beta_2))/(2*np.sqrt(6) + 1)
    condition_number_vec.append(condition_number)
    print(' Current normalized squared-root condition number is: ' + str(condition_number))
    print('sigma over gamma is ' + str(sigma_IHM_XTY/gamma_matrix_half_iter))

    ######################## AdaSSP ########################
    # AdaSSP base noise value: Gaussian mechanism for target of (\eps/3, \delta/3)
    sigma_base_AdaSSP_XTX   = calibrateAnalyticGaussianMechanism(eps/3.0, delta_DP/3.0, 1.0, tol = 1.e-12)
    sigma_base_AdaSSP_XTY   = calibrateAnalyticGaussianMechanism(eps/3.0, delta_DP/3.0, C_max, tol = 1.e-12)

    ######################## DP-GD ########################
    # DP-GD: Similar number of epochs as the IHM; Noise values taken from [Brown '24]
    rho_dpgd    = (np.sqrt(eps + np.log(1.0/delta_DP)) - np.sqrt(np.log(1.0/delta_DP)))**2
    sigma_dp_gd = np.sqrt(2.0 * iters_IHM * C_max**2 / rho_dpgd / (n**2))

    clipping_counter = 0.0

    # Monte-carlo run start here
    for iter in tqdm(range(iters)):

        ############################### Linear Mixing ###############################
        y_train_vec = y_train.reshape(-1, 1)  # (n,1)

        # Sample sketch and noises
        S = np.random.randn(k_val_LinMix, n)
        N_ours = np.random.randn(k_val_LinMix, d)
        N_ours_y = np.random.randn(k_val_LinMix)

        # Calculate minimal eigenvalue and noise level
        gamma_tilde = np.max((0, lambda_min_XY - sigma_eigenval * (tau - np.random.randn())))
        gamma_tilde = np.sqrt(np.max((0, gamma_matrix - gamma_tilde)))

        # Sketch X and Y
        X_train_full_PR = S @ X_train + gamma_tilde * N_ours
        y_PR = (S @ y_train_vec.reshape(-1, 1)).ravel() + gamma_tilde * N_ours_y

        # Solve the regression
        theta_LinMix = np.linalg.inv(X_train_full_PR.T @ X_train_full_PR) @ (X_train_full_PR.T @ y_PR)

        ############################### IHM with T = 1 ###############################

        # Sample sketch and noises
        S_iter = np.random.randn(k_val_IHM, n)
        N_ours_iter = np.random.randn(k_val_IHM, d)
        N_ours_y_iter = np.random.randn(k_val_IHM)

        # Calculate minimal eigenvalue and noise level
        gamma_tilde = np.max((0, lambda_min - sigma_eigenval_IHM_T_1 * (tau_mix - np.random.randn())))
        gamma_tilde = np.sqrt(np.max((0, sigma_IHM_T_1_X - gamma_tilde)))

        # Sketch X and add noise to XTY
        X_train_full_PR_hessian = S @ X_train + gamma_tilde * N_ours
        H_hat = (X_train_full_PR_hessian.T @ X_train_full_PR_hessian) / k_val_LinMix  # ≈ X^T X
        XY_Pr = X_train.T @ y_train + sigma_IHM_T_1_XTY * np.random.randn(d)

        theta_IHM_T_1 = np.linalg.solve(H_hat, XY_Pr)

        ############################### IHM with T = iters_IHM ###############################

        # Calculate minimal eigenvalue and noise level
        gamma_tilde = np.max((0, lambda_min - sigma_eigenval_half_iter * (tau_mix - np.random.randn())))
        gamma_final = np.sqrt(np.max((0, gamma_matrix_half_iter - gamma_tilde)))

        # Iterative loop
        theta_IHM_T_iter = np.zeros(d)
        for iter_hess in range(iters_IHM):

            # Sketch X and add noise to XTY
            S = np.random.randn(k_val_IHM, n)
            N_ours = np.random.randn(k_val_IHM, d)
            X_train_full_PR_hessian = S @ X_train + gamma_final * N_ours

            H_hat = (X_train_full_PR_hessian.T @ X_train_full_PR_hessian) / k_val_IHM  # ≈ X^T X
            XY_Pr = X_train.T @ np.clip(y_train - X_train @ theta_IHM_T_iter, -C_max, C_max) +\
                sigma_IHM_XTY * np.random.randn(d)

            # count the amount of iterations for which clipping occured
            if iter_hess > 0:
              if np.linalg.norm(np.clip(y_train - X_train @ theta_IHM_T_iter, -C_max, C_max) - (y_train - X_train @ theta_IHM_T_iter)) > 0:
                clipping_counter +=1

            # Update solution
            theta_IHM_T_iter += np.linalg.solve(H_hat, XY_Pr)

        ############################### AdaSSP ###############################

        # Calculate minimal eigenvalue and noise level
        lambda_min_tilde = np.max((0, lambda_min + sigma_base_AdaSSP_XTX * np.random.randn() - sigma_base_AdaSSP_XTX**2))
        lambda_adassp = np.max((0, np.sqrt(d * np.log(2.0*(d**2)/(target_varrho)))*sigma_base_AdaSSP_XTX - lambda_min_tilde))

        # Generate solution for AdaSSP
        N_upper = np.random.randn(d, d)
        N = np.triu(N_upper)  # Upper triangular part
        N_sym = N + N.T - np.diag(np.diag(N))  # Make symmetric
        XTX_noisy = X_train.T @ X_train + sigma_base_AdaSSP_XTX * N_sym
        XTY_noisy = X_train.T @ y_train + sigma_base_AdaSSP_XTY * np.random.randn(d)
        theta_adassp = np.linalg.inv(XTX_noisy + lambda_adassp*np.eye(d)) @ XTY_noisy

        ############################### DP_GD ###############################
        theta_DPGD = np.zeros(d)
        if 'DP_GD' in methods_to_plot:
            # Run DP-GD iterations
            for iter in range(iters_IHM):
                gt_clip = -1.0 * (1/n) * X_train.T @ np.clip(y_train - X_train @ theta_DPGD, -C_max, C_max)
                theta_DPGD += lr_dp_gd * (sigma_dp_gd * np.random.randn(d) - gt_clip)


        ############################### Evaluate ###############################
        # Evaluate on original test data
        y_test_pred = X_test @ theta_LinMix
        curr_test_mse_linear_mixing.append(np.mean((y_test - y_test_pred)**2))
        y_train_pred = X_train @ theta_LinMix
        curr_train_mse_linear_mixing.append(np.mean((y_train - y_train_pred)**2))

        y_test_pred = X_test @ theta_IHM_T_1
        curr_test_mse_IHM_T_1.append(np.mean((y_test - y_test_pred)**2))
        y_train_pred = X_train @ theta_IHM_T_1
        curr_train_mse_IHM_T_1.append(np.mean((y_train - y_train_pred)**2))

        y_test_pred = X_test @ theta_IHM_T_iter
        curr_test_mse_IHM.append(np.mean((y_test - y_test_pred)**2))
        y_train_pred = X_train @ theta_IHM_T_iter
        curr_train_mse_IHM.append(np.mean((y_train - y_train_pred)**2))

        y_test_pred = X_test @ theta_DPGD
        curr_test_mse_DP_GD.append(np.mean((y_test - y_test_pred)**2))
        y_train_pred = X_train @ theta_DPGD
        curr_train_mse_DP_GD.append(np.mean((y_train - y_train_pred)**2))

        y_test_pred = X_test @ theta_adassp
        curr_test_mse_adassp.append(np.mean((y_test - y_test_pred)**2))
        y_train_pred = X_train @ theta_adassp
        curr_train_mse_adassp.append(np.mean((y_train - y_train_pred)**2))

    # MSEs and confidence intervals
    mse_for_sigmas.append(np.mean(curr_test_mse_linear_mixing))
    mse_for_sigmas_std.append(1.96 * np.std(curr_test_mse_linear_mixing)/np.sqrt(iters))
    mse_for_sigmas_adassp.append(np.mean(curr_test_mse_adassp))
    mse_for_sigmas_std_adassp.append(1.96 * np.std(curr_test_mse_adassp)/np.sqrt(iters))
    mse_for_sigmas_IHM_T_1.append(np.mean(curr_test_mse_IHM_T_1))
    mse_for_sigmas_std_IHM_T_1.append(1.96 * np.std(curr_test_mse_IHM_T_1)/np.sqrt(iters))
    mse_for_sigmas_IHM.append(np.mean(curr_test_mse_IHM))
    mse_for_sigmas_std_IHM.append(1.96 * np.std(curr_test_mse_IHM)/np.sqrt(iters))
    mse_for_sigmas_DPGD.append(np.mean(curr_test_mse_DP_GD))
    mse_for_sigmas_std_DPGD.append(1.96 * np.std(curr_test_mse_DP_GD)/np.sqrt(iters))

    train_mse_for_sigmas.append(np.mean(curr_train_mse_linear_mixing))
    train_mse_for_sigmas_std.append(1.96 * np.std(curr_train_mse_linear_mixing)/np.sqrt(iters))
    train_mse_for_sigmas_adassp.append(np.mean(curr_train_mse_adassp))
    train_mse_for_sigmas_std_adassp.append(1.96 * np.std(curr_train_mse_adassp)/np.sqrt(iters))
    train_mse_for_sigmas_IHM_T_1.append(np.mean(curr_train_mse_IHM_T_1))
    train_mse_for_sigmas_std_IHM_T_1.append(1.96 * np.std(curr_train_mse_IHM_T_1)/np.sqrt(iters))
    train_mse_for_sigmas_IHM.append(np.mean(curr_train_mse_IHM))
    train_mse_for_sigmas_std_IHM.append(1.96 * np.std(curr_train_mse_IHM)/np.sqrt(iters))
    train_mse_for_sigmas_DPGD.append(np.mean(curr_train_mse_DP_GD))
    train_mse_for_sigmas_std_DPGD.append(1.96 * np.std(curr_train_mse_DP_GD)/np.sqrt(iters))

    print('====================')
    print('Test MSE: '          + str(np.mean(curr_test_mse_linear_mixing)))
    print('Test MSE ADASSP: '   + str(np.mean(curr_test_mse_adassp)))
    print('Test MSE Hessian T=1: '  + str(np.mean(curr_test_mse_IHM_T_1)))
    print('Test MSE Hessian '    + str(iters_IHM) + ' iters: ' + str(np.mean(curr_test_mse_IHM)))
    print('Test MSE DP GD: '    + str(np.mean(curr_test_mse_DP_GD)))

    print('====================')
    print('Train MSE: '         + str(np.mean(curr_train_mse_linear_mixing)))
    print('Train MSE ADASSP: '  + str(np.mean(curr_train_mse_adassp)))
    print('Train MSE Hessian: ' + str(np.mean(curr_train_mse_IHM_T_1)))
    print('Train MSE Hessian '   + str(iters_IHM) + ' iters: ' + str(np.mean(curr_train_mse_IHM)))
    print('Train MSE DP GD: '   + str(np.mean(curr_train_mse_DP_GD)))

    print('====================')
    print('Overall clipping events: ' + str(clipping_counter/iters/(iters_IHM - 1.0)))

    clipping_perc_vector.append(clipping_counter/iters/(iters_IHM - 1.0))

# Plotting Part

In [ ]:
################################# Generate Plots #################################
fig = plt.figure(figsize=(10, 7))
ax = plt.gca()

# Extract the single k value
label_suffix = r"$\frac{k}{\max\{d,\log(\frac{1}{\varrho})\}} = " + f"{percentage_k_Mix:.1f}$"
label_suffix_iter = r"$\frac{k}{\max\{d,\log(\frac{4T}{\varrho})\}} = " + f"{percentage_k_IHM:.1f}$"

# Define colors and markers for consistency
colors = {
    'adassp': 'blue',
    'linear_mixing': 'orangered',
    'hessian_mixing': 'green',
    'hessian_mixing_T_1': 'purple',
    'DPGD': 'black',
}
markers = {
    'adassp': '*',
    'linear_mixing': 'o',
    'hessian_mixing': 'D',
    'hessian_mixing_T_1': 'P',
    'DPGD': 'X',
}

eps = np.asarray(epsilon_values, dtype=float)
# Right subplot: Test MSE
if 'AdaSSP' in methods_to_plot:
    plt.plot(epsilon_values, train_mse_for_sigmas_adassp,
             color=colors['adassp'], marker=markers['adassp'], markersize=16,
             label="AdaSSP [Wang '18]")
    plt.fill_between(epsilon_values,
                     np.array(train_mse_for_sigmas_adassp) - np.array(train_mse_for_sigmas_std_adassp),
                     np.array(train_mse_for_sigmas_adassp) + np.array(train_mse_for_sigmas_std_adassp),
                     color=colors['adassp'], alpha=0.2)

if 'LinearMix' in methods_to_plot:
    plt.plot(epsilon_values, train_mse_for_sigmas,
             color=colors['linear_mixing'], marker=markers['linear_mixing'], markersize=16,
             label=fr"Linear mixing [Lev et al. '25], {label_suffix}")
    plt.fill_between(epsilon_values,
                     np.array(train_mse_for_sigmas) - np.array(train_mse_for_sigmas_std),
                     np.array(train_mse_for_sigmas) + np.array(train_mse_for_sigmas_std),
                     color=colors['linear_mixing'], alpha=0.2)

if 'DPGD' in methods_to_plot:
    plt.plot(epsilon_values, train_mse_for_sigmas_DPGD,
             color=colors['DPGD'], marker=markers['DPGD'], markersize=16,
             label=fr"DP-GD [Brown et al. '24]")
    plt.fill_between(epsilon_values,
                     np.array(train_mse_for_sigmas_DPGD) - np.array(train_mse_for_sigmas_std_DPGD),
                     np.array(train_mse_for_sigmas_DPGD) + np.array(train_mse_for_sigmas_std_DPGD),
                     color=colors['DPGD'], alpha=0.2)

if 'HessianMix' in methods_to_plot:
        # line (same color & linestyle), different marker per iteration
        label_iterative = (fr"IHM (ours), {iters_IHM} iters, {label_suffix_iter}")
        plt.plot(eps, train_mse_for_sigmas_IHM,
                 linestyle='--',
                 color=colors['hessian_mixing'],
                 marker=markers['hessian_mixing'],
                 markersize=16,
                 label=label_iterative)
        # condition number boxes
        # --- Add condition-number textboxes above each IHM point ---
        # Make sure len(condition_number_vector) == len(eps) == len(train_mse_for_sigmas_IHM)
        ax = plt.gca()
        for x, y, cn, cl in zip(eps, train_mse_for_sigmas_IHM, condition_number_vec, clipping_perc_vector):
            ax.annotate(
                rf"$\frac{{\chi \sqrt{{\kappa_{{X}}}}}}{{2 - \chi}}$: {cn:.2f}, clip: {cl*100:.0f}%",              # formatting; change as desired (e.g., :.1f)
                (x, y),
                textcoords="offset points",
                xytext=(0, 10),           # (dx, dy) in points; increase dy to move higher
                ha="center",
                va="bottom",
                fontsize=14,
                bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="none", alpha=0.75),
            )
        plt.fill_between(epsilon_values,
                     np.array(train_mse_for_sigmas_IHM) - np.array(train_mse_for_sigmas_std_IHM),
                     np.array(train_mse_for_sigmas_IHM) + np.array(train_mse_for_sigmas_std_IHM),
                     color=colors['hessian_mixing'], alpha=0.2)

if 'HessianMix_T_1' in methods_to_plot:
        # line (same color & linestyle), different marker per iteration
        label_iterative = (fr"IHM (ours): single iteration, {label_suffix}")
        plt.plot(eps, train_mse_for_sigmas_IHM_T_1,
                 linestyle='--',
                 color=colors['hessian_mixing_T_1'],
                 marker=markers['hessian_mixing_T_1'],
                 markersize=16,
                 label=label_iterative)
        plt.fill_between(epsilon_values,
                     np.array(train_mse_for_sigmas_IHM_T_1) - np.array(train_mse_for_sigmas_std_IHM_T_1),
                     np.array(train_mse_for_sigmas_IHM_T_1) + np.array(train_mse_for_sigmas_std_IHM_T_1),
                     color=colors['hessian_mixing_T_1'], alpha=0.2)

# Configure both subplots
plt.xlabel(r"$\epsilon_{\mathrm{DP}}$", fontsize=38)
plt.ylabel('Train MSE', fontsize=36)
plt.xscale('log')
plt.ticklabel_format(axis='y', scilimits=(-1, 1), useMathText=True)
plt.tick_params(axis='both', which='both', labelsize=36)
plt.grid(True)
ax = plt.gca()
ax.yaxis.get_offset_text().set_fontsize(36)
plt.tight_layout()

if add_legend_to_plot:
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.02), bbox_transform=plt.gcf().transFigure,  # anchor to figure
                    ncol=3, fontsize=22, frameon=False)

os.makedirs("plots", exist_ok=True)
plt.savefig(f"plots/hessian_mixing_manuscript_clipping_test_n_{n}_d_{d}_k_{percentage_k_Mix}.pdf", bbox_inches='tight')